# Train Emotion Recognition
ResMaskingNet which is available in PyFeat uses 142.9 million parameters with 74% accuracy.
Human emotion detection only reaches 65+-5%, poor golden standard

3-15 million

In [202]:
import wandb
import random
from torch import nn
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import math
import itertools
from sklearn.metrics import matthews_corrcoef as mcc

In [180]:
WANDB_API_KEY = "9ff9cdd1ffada991cc4f457217b7dfa6d0d7ce3b"
WANDB_NOTEBOOK_NAME = "Train Emotion Recognition"
wandb.login()

True

In [208]:
class MyNeuralNetwork(nn.Module):
    def __init__(self, layers, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(70, layers[0]),
            nn.LeakyReLU(),
            nn.Dropout(p = dropout[0]),
            nn.Linear(layers[0], layers[1]),
            nn.LeakyReLU(),
            nn.Dropout(p = dropout[1]),
            nn.Linear(layers[1], layers[2]),
            nn.LeakyReLU(),
            nn.Dropout(p = dropout[2]),
            nn.Linear(layers[2], layers[3]),
            nn.LeakyReLU(),
            nn.Dropout(p = dropout[3]),
            nn.Linear(layers[3], 7),
        )
    
    def forward(self, inputs):
        return self.net(inputs)

In [108]:
class MultiEmoVA(Dataset):
    def __init__(self, df):
        super().__init__()
        self.data = df
        self.inputs = torch.tensor(data.drop(["emotion","path", "index"], axis=1).to_numpy(np.float32))
        self.index2label = [label for label in data["emotion"].unique()]
        label2index = {label: i for i, label in enumerate(self.index2label)}
        self.labels = torch.tensor(data["emotion"].apply(lambda x: label2index[x]))

    
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, index):
        return self.inputs[index], self.labels[index]
    
    def map(self):
        return self.index2label


In [109]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [179]:
data = pd.read_csv("data\DiffusionFER\DiffusionEmotion_S\cropped\labels.csv")

learn, test = train_test_split(data, test_size = 0.1, stratify= data["emotion"])
train, val = train_test_split(learn, test_size = 0.2, stratify= learn["emotion"])

c:\Users\Shuai Shao\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Shuai Shao\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\Shuai Shao\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Shuai Shao\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and 

In [219]:
wandb.init(
    project = "Furhat Coach"
)

wandb.config = {
        "learning_rate": 0.00001,
        "architecture": (1024, 512, 256, 128),
        "epochs": 1000,
        "batch": 257,
        "dropout": (0, 0, 0.2, 0.3),
    }

###########################################################################################

train_loader = DataLoader(MultiEmoVA(train), batch_size= wandb.config["batch"], shuffle = True)
val_loader = DataLoader(MultiEmoVA(val), batch_size= len(val), shuffle = True)
test_loader = DataLoader(MultiEmoVA(test), batch_size= len(test), shuffle = True)

#model = MyNeuralNetwork(wandb.config["architecture"], wandb.config["dropout"]).to(device)
model = torch.load("models/acc_96.8")

def init_weights(layer):
    if isinstance(layer, nn.Linear):
        torch.nn.init.kaiming_normal_(layer.weight)

model.net.apply(init_weights)

loss_func = nn.CrossEntropyLoss()

optim = torch.optim.Adam(model.parameters(), lr = wandb.config["learning_rate"])


batch_size = 257

##########################################################################################

for epoch in range(wandb.config["epochs"]):
    model.train()
    train_correct = 0
    train_total = 0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        output = model(inputs)
        train_loss = loss_func(output, labels)
        train_loss.backward()
        optim.step()
        optim.zero_grad()
        train_correct += (output.softmax(dim=1).argmax(dim=1) == labels).sum()
        train_total += batch_size

    model.eval()
    val_correct = 0
    val_total = 0
    val_mcc = 0
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        output = model(inputs)
        val_loss = train_loss = loss_func(output, labels)
        val_correct += (output.softmax(dim=1).argmax(dim=1) == labels).sum()
        val_total += len(val)
        val_mcc = mcc(labels.to("cpu"), output.softmax(dim=1).argmax(dim=1).to("cpu"))
        
    print(f"Training accuracy is: {train_correct/train_total}")
    print(f"Validation accuracy is: {val_correct/val_total}")
    wandb.log({"train_acc": train_correct/train_total, "val_acc": val_correct/val_total,"train_loss": train_loss, "val_loss": val_loss, "val_mcc": val_mcc})

model.eval()
test_correct = 0
test_total = 0
test_mcc = 0
for inputs, labels, in test_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)
    output = model(inputs)
    test_correct += (output.softmax(dim=1).argmax(dim=1) == labels).sum()
    test_total += len(test)
    test_mcc = mcc(labels.to("cpu"), output.softmax(dim=1).argmax(dim=1).to("cpu"))

print(f"Test accuracy is: {test_correct/test_total}")
wandb.log({"test_acc": test_correct/test_total, "test_mcc": test_mcc})

wandb.finish()

Training accuracy is: 0.1424124538898468
Validation accuracy is: 0.2090517282485962
Training accuracy is: 0.16420233249664307
Validation accuracy is: 0.2492816150188446
Training accuracy is: 0.14785991609096527
Validation accuracy is: 0.25
Training accuracy is: 0.18210116028785706
Validation accuracy is: 0.2399425208568573
Training accuracy is: 0.1603112816810608
Validation accuracy is: 0.2198275923728943
Training accuracy is: 0.16887159645557404
Validation accuracy is: 0.1989942491054535
Training accuracy is: 0.16264592111110687
Validation accuracy is: 0.20545977354049683
Training accuracy is: 0.18365758657455444
Validation accuracy is: 0.2176724076271057
Training accuracy is: 0.1898832619190216
Validation accuracy is: 0.2277298867702484
Training accuracy is: 0.17976653575897217
Validation accuracy is: 0.24137930572032928
Training accuracy is: 0.18132296204566956
Validation accuracy is: 0.2492816150188446
Training accuracy is: 0.21867704391479492
Validation accuracy is: 0.256465524435

KeyboardInterrupt: 

In [ ]:
class ConfusionMatrix:
    def __init__(self, classlist):
        self.classes = {key: 0 for key in classlist}
        self.large_matrix = {key: 0 for key in itertools.product(classlist, classlist)}
        self.tn = 0
        self.fn = 0
        self.tp = 0
        self.fp = 0
    
    def add(label, result):
        if 
        

            

    

In [194]:
torch.save(model, "models/acc_96.8")
#torch.load

In [212]:
model = torch.load("models/acc_96.8")

In [213]:
wandb.init(
    project = "Furhat Coach"
)

In [207]:
def count_parameters(model): return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(model)

209287

In [218]:
test_correct = 0
test_total = 0
test_mcc = 0
for inputs, labels, in test_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)
    output = model(inputs)
    test_correct += (output.softmax(dim=1).argmax(dim=1) == labels).sum()
    test_total += len(test)
    test_mcc = mcc(labels.to("cpu"), output.softmax(dim=1).argmax(dim=1).to("cpu"))
    print(list(zip(output.softmax(dim=1).argmax(dim=1),labels)))
print(f"Test accuracy is: {test_correct/test_total}")


[(tensor(5, device='cuda:0'), tensor(5, device='cuda:0')), (tensor(6, device='cuda:0'), tensor(6, device='cuda:0')), (tensor(3, device='cuda:0'), tensor(3, device='cuda:0')), (tensor(1, device='cuda:0'), tensor(1, device='cuda:0')), (tensor(4, device='cuda:0'), tensor(4, device='cuda:0')), (tensor(6, device='cuda:0'), tensor(6, device='cuda:0')), (tensor(2, device='cuda:0'), tensor(2, device='cuda:0')), (tensor(6, device='cuda:0'), tensor(6, device='cuda:0')), (tensor(6, device='cuda:0'), tensor(6, device='cuda:0')), (tensor(3, device='cuda:0'), tensor(3, device='cuda:0')), (tensor(5, device='cuda:0'), tensor(5, device='cuda:0')), (tensor(0, device='cuda:0'), tensor(0, device='cuda:0')), (tensor(4, device='cuda:0'), tensor(4, device='cuda:0')), (tensor(6, device='cuda:0'), tensor(6, device='cuda:0')), (tensor(1, device='cuda:0'), tensor(1, device='cuda:0')), (tensor(4, device='cuda:0'), tensor(4, device='cuda:0')), (tensor(4, device='cuda:0'), tensor(4, device='cuda:0')), (tensor(3, de

In [ ]:
print(zip(output.softmax(dim=1).argmax(dim=1),labels))

In [215]:
wandb.log({"test_acc": test_correct/test_total, "test_mcc": test_mcc})
wandb.finish()

test_acc,▁
test_mcc,▁
test_acc,0.48405
test_mcc,0.98044
